<a href="https://colab.research.google.com/github/DinethRubeh/Cotton-Disease-Prediction/blob/master/Model_Training_ResNet50%2C_ResNet152V2%2C_VGG16%2C_InceptionV3_etc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [15]:
!nvidia-smi

Thu Oct  1 19:05:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    30W /  70W |    755MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [17]:
!pip install tensorflow==2.2.0

In [18]:
tf.__version__

'2.2.0'

In [19]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

# from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

import matplotlib.pyplot as plt

In [20]:
# Resize images
img_height = 224
img_width = 224
image_size = [img_height, img_width, 3] # height, width, depth

# Output classes
predict_class_size = 4

# batch size
bs = 32

# Train/test image path
train_dir = '/content/drive/My Drive/google colab/cotton_data/train'
test_dir = '/content/drive/My Drive/google colab/cotton_data/val'

In [21]:
model_dict = {VGG19:"vgg19", ResNet152V2:"resnet152v2", InceptionV3:"inceptionv3"}
# ResNet152V2:"resnet152v2", VGG19:"vgg19", InceptionV3:"inceptionv3"

In [22]:
def plot_loss_acc(model_history, model_name):
    
    ## Plot loss
    plt.plot(model_history.history['loss'], label='train loss')
    plt.plot(model_history.history['val_loss'], label='test loss')
    plt.legend()
    plt.savefig('/content/drive/My Drive/google colab/cotton_data/models/loss_{}'.format(model_name))
    plt.clf()

    ## Plot Accuracy
    plt.plot(model_history.history['accuracy'], label='train accuracy')
    plt.plot(model_history.history['val_accuracy'], label='test accuracy')
    plt.legend()
    plt.savefig('/content/drive/My Drive/google colab/cotton_data/models/accuracy_{}'.format(model_name))
    plt.clf()

In [23]:
def train_model(pre_trained_model):
    
    # Intialize pre trained model (ResNet152V2, VGG19, InceptionV3) with imagenet weights
    pre_trained_net = pre_trained_model(input_shape=image_size, weights='imagenet', include_top=False)

    # Freeze model weights (don't train)
    for layer in pre_trained_net.layers:
        layer.trainable = False

    # layers
    x = Flatten()(pre_trained_net.output)

    # output layer
    pred_layer = Dense(predict_class_size, activation='softmax')(x)
    
    # create model object
    model = Model(inputs=pre_trained_net.input, outputs=pred_layer)

    # Model structure
    # model.summary()

    # Set cost and optimization functions for model
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    # Image Data Augmentation using keras ImageDataGenerator
    # train data generator
    train_aug = ImageDataGenerator(rescale=1/255, 
                                   rotation_range=10, 
#                                    width_shift_range=0.2, # horizontal shift
#                                    height_shift_range=0.2, # vertical shift
#                                    horizontal_flip=True, 
#                                    vertical_flip=True, 
                                   zoom_range=0.2, # ~ [1-0.2, 1+0.2]
                                   brightness_range=[0.5,1.5]) # <1 darkens img, >1 brightens img
    # test data generator
    test_aug = ImageDataGenerator(rescale=1/255)

    # Create train/test set
    train_set = train_aug.flow_from_directory(train_dir, 
                                              target_size=(img_height,img_width), 
                                              batch_size=bs, 
                                              class_mode='categorical')

    test_set = test_aug.flow_from_directory(test_dir, 
                                            target_size=(img_height,img_width), 
                                            batch_size=bs, 
                                            class_mode='categorical')

    # fit the model
    hist = model.fit_generator(train_set, 
                              epochs=20,
                              steps_per_epoch=len(train_set),
                              validation_data=test_set,
                              validation_steps=len(test_set)
                             )
    
    # model name
    model_name = model_dict[pre_trained_model]
    
    # Save model loss/accuracy plots
    plot_loss_acc(hist,model_name)
    
    # Save model
    model.save('/content/drive/My Drive/google colab/cotton_data/models/model_{}.h5'.format(model_name))
    
    print("model {} trained and saved".format(model_name))

In [24]:
for each_model in list(model_dict.keys()):
    train_model(each_model)

80142336/80134624 [==============================] - 1s 0us/step
Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.
Epoch 1/20
61/61 [==============================] - 648s 11s/step - loss: 0.9215 - accuracy: 0.6873 - val_loss: 0.4847 - val_accuracy: 0.8364
Epoch 2/20
61/61 [==============================] - 41s 678ms/step - loss: 0.3229 - accuracy: 0.8821 - val_loss: 0.3013 - val_accuracy: 0.8765
Epoch 3/20
61/61 [==============================] - 42s 682ms/step - loss: 0.2446 - accuracy: 0.9134 - val_loss: 0.3123 - val_accuracy: 0.8735
Epoch 4/20
61/61 [==============================] - 42s 684ms/step - loss: 0.1897 - accuracy: 0.9344 - val_loss: 0.2115 - val_accuracy: 0.9198
Epoch 5/20
61/61 [==============================] - 42s 694ms/step - loss: 0.1800 - accuracy: 0.9390 - val_loss: 0.2977 - val_accuracy: 0.8981
Epoch 6/20
61/61 [==============================] - 42s 685ms/step - loss: 0.1440 - accuracy: 0.9559 - val_loss: 0.2260 - val_accuracy: 0.

<Figure size 432x288 with 0 Axes>